In [ ]:
#-- Matching Same Object --
#-- Predicting Direction ans Speed --
#-- Predicting Location of lost Objects --

In [ ]:
from IPython import display

In [ ]:
#-- Install ultralytics ------------------------------------------------------------------------------------------
!pip install ultralytics

display.clear_output()

import ultralytics
ultralytics.checks()
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Imports ----------------------------------------------------------------------------------------------------
from ultralytics import YOLO
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from collections import defaultdict
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
from datetime import datetime, timedelta
import shutil
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Initialize --------------------------------------------------------------------------------------------------
out_dir = '/kaggle/working/'
detection_weights_file = '/kaggle/input/yolo11-11frozen-13/model_11_frozen_epoch_60/train/weights/best.pt'

drone_files = ['/kaggle/input/drone-dataset-p1/v_5.mp4']#,
              #'/kaggle/input/drone-dataset-p2/v_8.mp4',
              #'/kaggle/input/drone-detection-test-videos-1/drone_video (1).mp4',
              # '/kaggle/input/novin-data/Novin_Dataset/f2.part2.mp4',
              #'/kaggle/input/sample-videos-detecting-and-matching-objs-1/sample_video_drone (5).mp4',
              #'/kaggle/input/video-drone-bird-1/Untitled-13.mp4']

results_dir = out_dir + 'results/'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

AREA_THRESHOLD = 5
DISTANCE_THRESHOLD = 50
SIMILARITY_THRESHOLD = 0.6
CROP_PADDING = 10
TIME_THRESHOLD = 60
NUM_TRACK_THRESHOLD = 30
DIRECTION_HISTORY = 10
MOVING_THRESHOLD = 1e-2
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Set Detection Model ------------------------------------------------------------------------------------------
model = YOLO(detection_weights_file)  
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Set Similarity Measure Model ---------------------------------------------------------------------------------
similarity_base_model = ResNet50(weights='imagenet')

#-- Use the second-last layer for embeddings --
similarity_model = Model(inputs=similarity_base_model.input,
                         outputs=similarity_base_model.layers[-2].output)  
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to Preprocess Image for Similarity Measure ---------------------------------------------------------
def preprocess_image(image, target_size=(224, 224)):
    
    image = cv2.resize(image, target_size) 
    image = np.expand_dims(image, axis=0)  #-- Add batch dimension
    image = preprocess_input(image)  #-- Normalize for ResNet
    return image
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to Calculate Similarity -----------------------------------------------------------------------------
def compare_similarity_images(image1, image2):   
    
    #-- Preprocess images --
    img1 = preprocess_image(image1)
    img2 = preprocess_image(image2)
    
    #-- Extract features --
    embedding1 = similarity_model.predict(img1)
    embedding2 = similarity_model.predict(img2)

    #-- Compute cosine similarity --
    similarity_score = cosine_similarity(embedding1, embedding2)[0][0]     

    return similarity_score
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to Match Detected Objects ---------------------------------------------------------------------------
def match_object(track_id, track_box, track_image, track_time, last_tracked_objects):   
    
    # plt.imshow(track_image)
    # plt.title(f'track_object - id:{track_id}')
    # plt.axis('off')  
    # plt.show()    
    
    track_center_x, track_center_y, track_w, track_h = track_box     
    
    distance_match = {}
    similarity_match = {}
    
    matched_id = None
    
    for obj_id, (obj_box, obj_img, obj_time) in last_tracked_objects.items():
        
        print(f'####################### {obj_id} #####################')
        # plt.imshow(obj_img)
        # plt.title(f'object- id:{obj_id}')
        # plt.axis('off')  
        # plt.show()    
        
        time_difference = abs(track_time - obj_time)
        print(f'-------- time_difference: {time_difference} -------------')
        if time_difference > timedelta(seconds=TIME_THRESHOLD):            
            continue
        
        similarity_score = compare_similarity_images(track_image, obj_img)
        print(f'-------- similarity_score: {similarity_score} -------------')
        if similarity_score < SIMILARITY_THRESHOLD:            
            continue
        
        x_center, y_center, w, h = obj_box     
        
        a_track = track_w * track_h
        a_obj = w *h        
        if a_track>a_obj:
            a_ratio = a_track/a_obj
        else:
            a_ratio = a_obj/a_track
        
        print(f'-------- a_ratio: {a_ratio} -------------')
        if a_ratio > AREA_THRESHOLD:            
            continue         
        
        distance = np.sqrt((track_center_x - x_center)**2 + (track_center_y - y_center)**2)      
        print(f'-------- distance: {distance} -------------')
        if distance <= DISTANCE_THRESHOLD:            
            distance_match[obj_id] = distance
        else:
            similarity_match[obj_id] = similarity_score
    
    if len(distance_match)!=0:
        matched_id = min(distance_match, key=distance_match.get) 
    elif len(similarity_match)!=0:
        matched_id = min(similarity_match, key=similarity_match.get)       
    
    print(f'-------- matched_id: {matched_id} -------------')
    return matched_id

      
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function To Crop Object Using Bounding Box ---------------------------------------------------------------
def crop_object(frame, box, padding=CROP_PADDING):
    
    frame_height, frame_width = frame.shape[:2]    
    center_x, center_y, w, h = box  
    
    top_left_x = int(max(center_x - w // 2 - padding, 0))
    top_left_y = int(max(center_y - h // 2 - padding, 0))
    bottom_right_x = int(min(center_x + w // 2 + padding, frame_width))
    bottom_right_y = int(min(center_y + h // 2 + padding, frame_height))
    
    cropped_object = frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x].copy()
    
    return cropped_object
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
def predict_direction(track_history, track_id, num_frames=DIRECTION_HISTORY, threshold=MOVING_THRESHOLD):
    track = track_history.get(track_id, [])
    if len(track) < num_frames:
        return None, (0, 0)  # Not enough data to predict direction  

    # Check if object is stationary
    deltas = []
    for i in range(1, len(track)):
        delta_x = track[i][0] - track[i-1][0]
        delta_y = track[i][1] - track[i-1][1]
        deltas.append(np.sqrt(delta_x**2 + delta_y**2))  # Euclidean distance

    avg_delta = np.mean(deltas[-num_frames:])
    if avg_delta < 5:
        return "Stationary", (0, 0)  
    
    # Predict Direction
    delta_x = track[-1][0] - track[-num_frames][0]
    delta_y = track[-1][1] - track[-num_frames][1]

    # Normalize the direction vector
    magnitude = np.sqrt(delta_x**2 + delta_y**2)
    if magnitude > 0:
        delta_x /= magnitude
        delta_y /= magnitude

    if abs(delta_x) < threshold and abs(delta_y) < threshold:
        return "Stationary", (0, 0)

    directions = {
        (-1, -1): "Up-Left", (-1, 0): "Left", (-1, 1): "Down-Left",
        (0, -1): "Up", (0, 0): "Stationary", (0, 1): "Down",
        (1, -1): "Up-Right", (1, 0): "Right", (1, 1): "Down-Right",
    }
    dx_sign = np.sign(delta_x)
    dy_sign = np.sign(delta_y)
    direction = directions.get((dx_sign, dy_sign), "Unknown")
    return direction, (delta_x, delta_y)

In [ ]:
#-- Function to Calculate Speed of Movement --------------------------------------------------------------------all
def calculate_speed(track_history, track_id, fps):
    
    track = track_history.get(track_id, [])
    if len(track) >= 2:  #-- At least two points are required
        x1, y1 = track[-2]  #-- Previous coordinates
        x2, y2 = track[-1]  #-- Current coordinates
        
        distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
        
        speed = distance #* fps  #-- Pixels per second
        return speed
    return 0  
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to Calculate Arrow of Direction ------------------------------------------------------------------
def get_arrow(x, y, delta_x, delta_y):
    
    start_point = (int(x), int(y))  #-- center of bounding box                             
                            
    arrow_scale = 50 
    normalized_dx = delta_x / max(abs(delta_x), abs(delta_y), 1e-5)  
    normalized_dy = delta_y / max(abs(delta_x), abs(delta_y), 1e-5)                            
                            
    end_point = (int(x + normalized_dx * arrow_scale),
                 int(y + normalized_dy * arrow_scale))

    return start_point, end_point
#--------------------------------------------------------------------------------------------------------------

In [ ]:
def is_object_exiting(frame_width, frame_height, bbox, direction_vector, margin=10):
    
    x_min, y_min, x_max, y_max = bbox
    delta_x, delta_y = direction_vector
    
    
    near_left = x_min <= margin
    near_right = x_max >= frame_width - margin
    near_top = y_min <= margin
    near_bottom = y_max >= frame_height - margin
    
    
    moving_left = delta_x < 0 and near_left
    moving_right = delta_x > 0 and near_right
    moving_up = delta_y < 0 and near_top
    moving_down = delta_y > 0 and near_bottom
    
    
    return moving_left or moving_right or moving_up or moving_down


In [ ]:
def update_bbox_position(bbox, speed, delta_x, delta_y):
    x, y, w, h = bbox

    normalized_dx = delta_x / max(abs(delta_x), abs(delta_y), 1e-5)  
    normalized_dy = delta_y / max(abs(delta_x), abs(delta_y), 1e-5)   
    
    # Calculate the new position based on velocity and direction
    # new_x = x + (speed/10) * normalized_dx
    # new_y = y + (speed/10) * normalized_dy

    speed = speed/3
    new_x = x + speed * normalized_dx
    new_y = y + speed * normalized_dy

    # Return the updated bounding box
    return new_x, new_y, w, h


In [ ]:
#-- Run ----------------------------------------------------------------------------------------------------------
for video_file in drone_files:
    
    #-- get video name --
    index = video_file.rfind('/')      
    video_name = video_file[index + 1:] 
    
    #-- set output file --
    out_video_name = 'out_' + video_name    
    output_path = results_dir + out_video_name
    
    print(f'=== Processing {video_name} ================================')
    
    cap = cv2.VideoCapture(video_file)
    
    #-- get video properties --
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    #-- set video writer --
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    track_history = defaultdict(lambda: []) #-- for tracking
    last_tracked_objects = {} #-- for matching  
    last_pred_objects = {} 
    last_ids = set() #-- set of last ids     
    last_status = {} #-- 
    last_speed = {}
    mapped_objects = {}
    frame_number = 0
    
    while cap.isOpened():    
        success, frame = cap.read()
        if success:            
            frame_number += 1
            print(f'\nframe number = {frame_number} ==================================') 
            print('track_history:', track_history.keys())
            print('last_tracked_objects:', last_tracked_objects.keys())

            for k,v in last_tracked_objects.items():
                print(f'----{k}-{v[0]} ')

            
            # if frame_number>=1000:
            #     break            

            current_ids = set() #-- set of ids in the cuurent framse
            
            #-- detect and track objects --
            results = model.track(frame,    
                                  tracker = 'bytetrack.yaml',
                                  persist=True,
                                  show = False)

            #-- Check if there are any detections --
            if results[0].boxes is not None and results[0].boxes.xywh is not None:            
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id
                if track_ids is not None:
                    track_ids = track_ids.int().cpu().tolist()       
                                
                    for box, track_id in zip(boxes, track_ids):     

                        #-- Crop the object from the frame --
                        cropped_object = crop_object(frame, box)
                        detection_time = datetime.now()
                        
                        #-- Check if this is the first detected object --
                        if len(last_tracked_objects)==0:
                            last_tracked_objects[track_id] = (box, cropped_object, detection_time)
                        else:
                            #-- Check if its not new detected object --
                            if track_id in last_tracked_objects:
                                last_tracked_objects[track_id] = (box, cropped_object, detection_time)                   
                                
                            else:   
                                if track_id in mapped_objects:
                                    matched_id =  mapped_objects[track_id]
                                else:
                                    matched_id = match_object(track_id, box, cropped_object, detection_time, last_tracked_objects)
                                if matched_id is not None:
                                    mapped_objects[track_id] = matched_id
                                    track_id = matched_id                                    
                                last_tracked_objects[track_id] = (box , cropped_object, detection_time)   
                                
                        
                        current_ids.add(track_id)
                        last_ids.add(track_id)

                        if track_id in last_pred_objects:
                            del last_pred_objects[track_id]
                        
                        
                                
                        #-- track --                  
                        annotated_frame = frame
                        x, y, w, h = box
                        track = track_history[track_id]
                        track.append((float(x), float(y)))  #-- x, y center point
                        if len(track) > NUM_TRACK_THRESHOLD:  #-- retain NUM_TRACK_THRESHOLD tracks
                            track.pop(0)

                        #-- Draw the tracking lines --
                        points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                        cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 0, 255), thickness=4)
                        
                        #-- Draw the bounding box --
                        top_left = (int(x - w / 2), int(y - h / 2))
                        bottom_right = (int(x + w / 2), int(y + h / 2))
                        cv2.rectangle(annotated_frame, top_left, bottom_right, (255, 0, 0), 2)  #-- Blue bounding box

                        #-- Put the ID text --
                        text_position = (int(x - w / 2), int(y - h / 2) - 10)
                        cv2.putText(annotated_frame, f'ID: {track_id}', text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                        #-- Calculate Speed and Direction --
                        speed = calculate_speed(track_history, track_id, fps)
                        direction, (delta_x, delta_y) = predict_direction(track_history, track_id)

                        last_speed[track_id] = (speed, (delta_x, delta_y))

                        if direction != "Stationary": 
                            last_status[track_id] = is_object_exiting(frame_width,
                                                                  frame_height,
                                                                  box,
                                                                  (delta_x, delta_y))
                        else:
                            last_status[track_id] = False

                        #-- Calculating the coordinates of the bounding box --
                        top_left_x = int(x - w / 2)
                        top_left_y = int(y - h / 2)
                        bottom_right_x = int(x + w / 2)
                        bottom_right_y = int(y + h / 2)

                        if direction == "Stationary":                            
                            text_position = (top_left_x, bottom_right_y + 20)  
                            cv2.putText(frame, direction, text_position, 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        else:
                            start_point, end_point = get_arrow(x, y, delta_x, delta_y)
                            
                            #-- Draw Arrow on the frame --
                            cv2.arrowedLine(annotated_frame, start_point, end_point, (255, 0, 0), 2, tipLength=0.5)
                    
                            #-- Put the direction and speed under the bounding box --
                            text_position = (top_left_x, bottom_right_y + 20)  
                            cv2.putText(frame, f'Dir: {direction}, Speed: {speed:.2f}px/s', text_position, 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)                           
                        
                else:
                    annotated_frame = frame  #-- If no IDs, use original frame

            else:
                annotated_frame = frame  #-- If no boxes, use original frame

            print('############################################')
            for k,v in last_tracked_objects.items():
                print(f'----{k}-{v[0]} ')
            print('############################################')
            
            for id in last_ids:
                # if id not in current_ids and last_status[id]:
                #     x , y , w, h = last_tracked_objects[id][0] #-- box                    
                #     top_left = (int(x - w / 2), int(y - h / 2))
                #     bottom_right = (int(x + w / 2), int(y + h / 2))
                #     cv2.rectangle(annotated_frame, top_left, bottom_right, (255, 255, 255), 2)  

                
                    
                    
                if id not in current_ids and not last_status[id]:
                    if id not in last_pred_objects:
                        last_box = last_tracked_objects[id][0]
                    else:
                        last_box = last_pred_objects[id][0]                        
                        
                    new_box = update_bbox_position(bbox= last_box,
                                                   speed= last_speed[id][0],
                                                   delta_x= last_speed[id][1][0],
                                                   delta_y= last_speed[id][1][1])
                    print('**************************', id)
                    print('last_box:' , last_box)
                    print('pred_box:' , new_box)
                    print('*****************************************')

                    cropped_object = last_tracked_objects[id][0]
                    detection_time = datetime.now()
                    last_pred_objects[id] = (new_box, cropped_object, detection_time)
                    
                    #-- Draw the bounding box --
                    #x , y , w, h = last_tracked_objects[id][0] #-- box
                    x , y , w, h = new_box
                    top_left = (int(x - w / 2), int(y - h / 2))
                    bottom_right = (int(x + w / 2), int(y + h / 2))
                    cv2.rectangle(annotated_frame, top_left, bottom_right, (255, 255, 0), 2)  

                    #-- track --  
                    track = track_history[id]
                    track.append((float(x), float(y)))  #-- x, y center point
                    if len(track) > NUM_TRACK_THRESHOLD:  #-- retain NUM_TRACK_THRESHOLD tracks
                        track.pop(0)

                    #-- Draw the tracking lines --
                    points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(255, 255, 0), thickness=4)  

                    #-- Put the ID text --
                    text_position = (int(x - w / 2), int(y - h / 2) - 10)
                    cv2.putText(annotated_frame, f'ID: {id}', text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                                  

                    
                    

            out.write(annotated_frame)

        else:
            #-- Break the loop if the end of the video is reached --
            break

    #-- Release the video capture object and close the display window --
    cap.release()
    out.release()   
    #display.clear_output()
    

In [ ]:
# zip_results = "results"
# shutil.make_archive(zip_results, 'zip', results_dir)